In [9]:
import requests
from pygltflib import GLTF2
from io import BytesIO
import tempfile
import numpy as np
from PIL import Image
import base64
import pandas as pd
import os

# Function to fetch the GLB file content
def fetch_glb_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        raise Exception(f"Failed to fetch file: {response.status_code}")

# Function to load and parse the GLB file from content
def load_glb_from_content(content):
    with tempfile.NamedTemporaryFile(delete=True, suffix=".glb") as temp_file:
        temp_file.write(content.read())
        temp_file.flush()
        gltf = GLTF2().load_binary(temp_file.name)
    return gltf

# Function to extract textures and colors from the GLB file
def extract_textures_and_colors(gltf):
    textures = []
    for material in gltf.materials:
        if material.pbrMetallicRoughness.baseColorTexture is not None:
            textures.append(material.pbrMetallicRoughness.baseColorTexture.index)
    return textures

# Function to decode base64 data URIs
def decode_base64_data_uri(data_uri):
    header, encoded = data_uri.split(",", 1)
    data = base64.b64decode(encoded)
    return data

# Function to save texture images
def save_texture_images(gltf):
    texture_images = []
    for i, texture in enumerate(gltf.textures):
        image_index = texture.source
        image = gltf.images[image_index]
        
        if image.uri:
            # Decode the base64 data URI
            image_data = decode_base64_data_uri(image.uri)
        else:
            # Handle embedded binary data
            buffer_view = gltf.bufferViews[image.bufferView]
            buffer = gltf.buffers[buffer_view.buffer]
            data = gltf.get_data_from_buffer_uri(buffer.uri)
            start = buffer_view.byteOffset
            end = start + buffer_view.byteLength
            image_data = data[start:end]

        # Save the image
        image_path = f'texture_{i}.png'
        with open(image_path, 'wb') as f:
            f.write(image_data)
        texture_images.append(image_path)
    return texture_images

# Dummy function to compare two images (You can replace this with your actual comparison logic)
def compare_images(image1, image2):
    # Example: Use mean squared error (MSE) for comparison
    image1 = image1.resize((100, 100)).convert('L')  # Resize and convert to grayscale
    image2 = image2.resize((100, 100)).convert('L')data = {'img': [os.path.join(image_dir, img) for img in image_files]}
    
    arr1 = np.array(image1)
    arr2 = np.array(image2)
    
    mse = np.mean((arr1 - arr2) ** 2)
    return mse

# Function to fetch the image from a given path
def fetch_image_from_path(image_path):
    try:
        return Image.open(image_path)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Example usage
glb_url = "https://models.readyplayer.me/66a54feb8409082e90051729.glb"

# Fetch the GLB content
glb_content = fetch_glb_content(glb_url)

# Load and parse the GLB file from content
gltf = load_glb_from_content(glb_content)

# Extract textures
texture_image_paths = save_texture_images(gltf)

# Paths to extracted texture images
extracted_texture_images = [fetch_image_from_path(img_path) for img_path in texture_image_paths]

# Load dataset
dataset_csv = 'image_dataset.csv'  # Update with the correct path to your CSV file
df = pd.read_csv(dataset_csv)

best_score = float('inf')
best_match = None

# Compare each image in the dataset with the extracted texture images
for index, row in df.iterrows():
    try:
        image_path = row['img']  # Update with the correct column name in your CSV
        image = fetch_image_from_path(image_path)
        if image is not None:
            for extracted_texture in extracted_texture_images:
                similarity_score = compare_images(extracted_texture, image)
                if similarity_score < best_score:
                    best_score = similarity_score
                    best_match = image_path
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Output the best match
print("Best match found:")
print(f"Image: {best_match}")
print(f"Similarity Score: {best_score}")

Best match found:
Image: /home/vanshika/Desktop/images/image.jpeg
Similarity Score: 83.794
